In [72]:
import icalendar
import uuid
from datetime import datetime, timezone
import numpy as np, pandas as pd
import os

def format_timestamp(datetime_obj: datetime) -> str:

    return datetime_obj.strftime('%Y%m%dT%H%M%SZ')

def create_new_event(start: datetime, end: datetime, event_name: str, description: str, location: str, uid: str) -> icalendar.Event:

    event: icalendar.Event = icalendar.Event()
    current_time: datetime = datetime.now(timezone.utc)
    event.add('transp', 'TRANSPARENT')
    event.add('summary', event_name)
    event.add('description', description)
    event.add('location', location)    
    event.add('uid', uid)

    event['dtstart'] = format_timestamp(start)
    event['dtend'] = format_timestamp(end)
    event['created'] = format_timestamp(current_time)
    event['last-modified'] = format_timestamp(current_time)
    event['dtstamp'] = format_timestamp(current_time)

    return event

def create_ical_file(ics_file, sparkle_file, input_file, locationprefix="", timezone="", eventType=""):

    df = pd.read_csv(input_file, keep_default_na=False)
    cal: icalendar.Calendar = icalendar.Calendar()
    df_sparkle = pd.DataFrame(columns = ['location','name','host','description','start','duration','eventId'])
    
    for index, row in df.iterrows():    
        if eventType in row['Location']:
            if timezone in row['TimeZoneBlock']:
                start: datetime = datetime.strptime(str(row['Date Start']) + 'T' + str(row['Time Start']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ')                        
                start_sparkle: int = int(datetime.strptime(str(row['Date Start']) + 'T' + str(row['Time Start']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ').timestamp())
                duration: str = str(int((datetime.strptime(str(row['Date End']) + 'T' + str(row['Time End']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ') - datetime.strptime(str(row['Date Start']) + 'T' + str(row['Time Start']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ')).total_seconds()/60))
                end: datetime = datetime.strptime(str(row['Date End']) + 'T' + str(row['Time End']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ')
                event_name: str = row['Title']
                # set based on available data:
                if row['Speaker'] and row['Moderator']:
                    if "Student and Postdoc SIG Workshop" in row['Moderator']:
                        description: str = row['Categories'] + '\n' + row['Moderator'] + '\nSpeaker: ' + row['Speaker']
                        description_sparkle: str = row['Categories'] + ', ' + row['Moderator'] + ', Speaker: ' + row['Speaker']
                    else:
                        description: str = row['Categories'] + '\nSpeaker: ' + row['Speaker'] + '\nModerator: ' + row['Moderator']
                        description_sparkle: str = row['Categories'] + ', Speaker: ' + row['Speaker'] + ', Moderator: ' + row['Moderator']
                elif row['Speaker']:
                    description: str = row['Categories'] + '\nSpeaker: ' + row['Speaker']
                    description_sparkle: str = row['Categories'] #+ ', Speaker: ' + row['Speaker']
                elif row['Moderator']:                    
                    description: str = row['Categories'] + '\n' + row['Moderator']
                    description_sparkle: str = row['Categories'] + ', ' + row['Moderator']
                else:                    
                    description: str = row['Categories']
                    description_sparkle: str = row['Categories']

                if row['Speaker'] and row['Moderator']:
                    host: str = row['Speaker'] + ", Moderated by " + row['Moderator']
                elif row['Speaker']:
                    host: str = row['Speaker']
                elif row['Moderator']:
                    host: str = row['Moderator']
                else:
                    host: str = ""

                location: str = 'https://ohbm.sparkle.space/in/' + locationprefix + row['Location'].replace('-','')
                locationName: str = locationprefix + row['Location'].replace('-','')

                # set UID for event 
                if row['UID']:
                    uid: str = row['UID']
                else:
                    uid: str = str(uuid.uuid1())         
                df.at[index,'UID'] = uid

                if row['EVENTID']:
                    eventId: str = row['EVENTID']
                else:
                    eventId: str = ""

                df_sparkle = df_sparkle.append({'location': locationName, 
                                                'name': event_name, 
                                                'host': host,
                                                'description': description_sparkle, 
                                                'start': start_sparkle,
                                                'duration': duration,
                                                'eventId': eventId
                                               }, ignore_index=True)

                event: icalendar.Event = create_new_event(start, end, event_name, description, location, uid)
                cal.add_component(event)

    df.to_csv(input_file, index=False)
    df_sparkle.to_csv(sparkle_file, index=False)    
    
    with open(ics_file, 'wb') as o:
        o.write(cal.to_ical())

In [73]:
create_ical_file('calendars/conference/OHBM_Schedule_All.ics', './OHBM2021_schedule_for_sparkle.csv', 'OHBM2021_schedule.csv')

In [29]:
df = pd.read_csv('hbm_educational_schedule_2021.csv', keep_default_na=False)
TZs = df['TimeZoneBlock'].unique()
LOCs = df['Location'].unique()

for tz in TZs:
    for loc in LOCs:
        create_ical_file('calendars/educationalCourses/OHBM2021_EduCourses_Schedule_%s_%s.ics' % (tz,loc.replace('-','')), './hbm_educational_schedule_2021_for_sparkle_%s_%s.csv' % (tz,loc), 'hbm_educational_schedule_2021.csv', locationprefix="course", timezone=tz, eventType=loc)
        
create_ical_file('calendars/educationalCourses/OHBM2021_EduCourses_Schedule_All.ics', './hbm_educational_schedule_2021_for_sparkle.csv', 'hbm_educational_schedule_2021.csv', locationprefix="course")

In [70]:
def create_ical_file_posters(ics_file, df_poster):

    cal: icalendar.Calendar = icalendar.Calendar()
    
    for index, row in df_poster.iterrows():    
        start: datetime = datetime.strptime(str(row['Date Start']) + 'T' + str(row['Time Start']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ')                        
        start_sparkle: int = int(datetime.strptime(str(row['Date Start']) + 'T' + str(row['Time Start']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ').timestamp())
        duration: str = str(int((datetime.strptime(str(row['Date End']) + 'T' + str(row['Time End']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ') - datetime.strptime(str(row['Date Start']) + 'T' + str(row['Time Start']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ')).total_seconds()/60))
        end: datetime = datetime.strptime(str(row['Date End']) + 'T' + str(row['Time End']).zfill(6) + 'Z', '%Y%m%dT%H%M%SZ')
        event_name: str = row['Title']
        description: str = row['Categories'] + '\n' + row['Speaker']            
        # set UID for event 
        if row['UID']:
            uid: str = row['UID']
        else:
            uid: str = str(uuid.uuid1())         
        df.at[index,'UID'] = uid
        location: str = 'https://ohbm.sparkle.space/in/' + row['Location']

        event: icalendar.Event = create_new_event(start, end, event_name, description, location, uid)
        cal.add_component(event)
    
    with open(ics_file, 'wb') as o:
        o.write(cal.to_ical())

In [71]:
df = pd.read_csv('OHBM2021_schedule_posters_all.csv', keep_default_na=False)
PSTRs = df['Submission No'].unique()

for p in PSTRs:
    create_ical_file_posters('calendars/posters/poster%s.ics' % str(p), df[df['Submission No'] == p])